# QA tests on FiberFlat (v1.0 by JXP)

## Load 'true' model spectrum

In [1]:
# imports
import specter.throughput as spec_thru
import specter.psf.spotgrid as SpotGridPSF
sys.path.append(os.path.abspath("/Users/xavier/DESI/desisim_v0.4.1/desisim/"))
import interpolation as desi_interp
import io as desisim_io

In [33]:
# Read flat
infile = '/Users/xavier/DESI/CALIBS/flat-3100K-quartz-iodine.fits'
hdu = fits.open(infile)
hdu.info()

Filename: /Users/xavier/DESI/CALIBS/flat-3100K-quartz-iodine.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      11   (67461,)     float64   


In [44]:
flux = hdu[0].data
hdr = hdu[0].header
# from desisim_io
wave = hdr['CRVAL1'] + np.arange(hdr['NAXIS1'])*hdr['CDELT1']
if hdr['LOGLAM'] == 1:
    wave = 10**wave

In [45]:
#- resample to 0.2 A grid
dw = 0.6
ww = np.arange(3579., 5938.8, dw)
flux = desi_interp.resample_flux(ww, wave, flux)

In [5]:
xdb.xplot(ww,flux)

## Apply thruput

In [ ]:
thru = spec_thru.load_throughput('/Users/xavier/DESI/desimodel/data/throughput/thru-b.fits')

In [46]:
# Load PSF
psf_fil = '/Users/xavier/DESI/desimodel/data/specpsf/psf-b.fits'
psf = SpotGridPSF.SpotGridPSF(psf_fil)

In [47]:
# Generate photons from thruput
ii = (psf.wmin <= ww) & (ww <= psf.wmax)
phot = thru.photons(ww[ii], flux[ii], units=hdr['BUNIT'], objtype='CALIB')

In [39]:
# Plot
xdb.xplot(ww[ii],phot)

## Write to file

In [10]:
from linetools.spectra.xspectrum1d import XSpectrum1D

In [48]:
xspec = XSpectrum1D.from_tuple((ww[ii],phot))

In [49]:
xspec.write_to_fits('/Users/xavier/DESI/CALIBS/b-flat-phot_spec.fits')

Wrote spectrum to /Users/xavier/DESI/CALIBS/b-flat-phot_spec.fits


## Now fiddle about in desispec.fiberflat

In [20]:
#imports
from desispec.io import frame as desi_io_frame
from desispec import fiberflat as dspec_fflat

### Added make_qa method (not part of Class yet)

In [18]:
path = '/Users/xavier/DESI/TST/20150211/'

In [19]:
# Load
flat_frame = desi_io_frame.read_frame(path+'frame-b0-00000001.fits')

In [21]:
# Process + checking
reload(dspec_fflat)
fiber_flat = dspec_fflat.compute_fiberflat(flat_frame)

INFO:fiberflat.py:51:compute_fiberflat: starting


INFO:DESI:starting


INFO:fiberflat.py:111:compute_fiberflat: iter 0 fiber 0


INFO:DESI:iter 0 fiber 0


INFO:fiberflat.py:111:compute_fiberflat: iter 0 fiber 10


...

INFO:DESI:rejecting


INFO:fiberflat.py:179:compute_fiberflat: iter #7 chi2=110010.287774 ndf=73637 chi2pdf=1.493954 nout=0


INFO:DESI:iter #7 chi2=110010.287774 ndf=73637 chi2pdf=1.493954 nout=0


INFO:fiberflat.py:191:compute_fiberflat: nout tot=322


INFO:DESI:nout tot=322


In [62]:
# Had to scale the two to match.  Unsure why
reload(dspec_fflat)
dspec_fflat.make_qa(fiber_flat.wave, fiber_flat.meanspec)